# Markov Decision Processes 
This notebook reproduces the key concepts and the "Student" examples from David Silver's Lecture 2.

Sections:
1. Markov Processes (MP)
2. Markov Reward Processes (MRP)
3. Markov Decision Processes (MDP)

Run cells in order. Set kernel to Python 3 if needed.

In [1]:
# Section 1: Setup & Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## Section 1: Markov Chain (MP)

A **Markov Process** (or **Markov Chain**) is a tuple ⟨𝒮, ℙ⟩:

- **𝒮** is a (finite) set of states  
- **ℙ** is a state transition probability matrix,  
  ℙₛₛ′ = ℙ[𝑆ₜ₊₁ = s′ | 𝑆ₜ = s]


### Student Example
This section defines the Student Markov Chain transition matrix and demonstrates sampling episodes (random walk).

![alt text](images/student_example_1.png)


In [ ]:
# Define states and transition matrix P
states = ["Class 1", "Class 2", "Class 3", "Pass", "Pub", "Facebook", "Sleep"]
n_states = len(states)

P = np.array([
    [0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0],  # Class 1
    [0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.2],  # Class 2
    [0.0, 0.0, 0.0, 0.6, 0.4, 0.0, 0.0],  # Class 3
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],  # Pass
    [0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0],  # Pub
    [0.1, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0],  # Facebook
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]   # Sleep
])

# Display
print("State Transition Matrix P:")
df_P = pd.DataFrame(P, index=states, columns=states)
display(df_P)

# Validate rows sum to 1
row_sums = P.sum(axis=1)
print("\nRow sums (should be 1.0 for each state):")
print(pd.Series(row_sums, index=states))
print("\nAll rows sum to 1.0:", np.allclose(row_sums, 1.0))

State Transition Matrix P:


,Class 1,Class 2,Class 3,Pass,Pub,Facebook,Sleep
Class 1,0.0,0.5,0.0,0.0,0.0,0.5,0.0
Class 2,0.0,0.0,0.8,0.0,0.0,0.0,0.2
Class 3,0.0,0.0,0.0,0.6,0.4,0.0,0.0
Pass,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Pub,0.2,0.4,0.4,0.0,0.0,0.0,0.0
Facebook,0.1,0.0,0.0,0.0,0.0,0.9,0.0
Sleep,0.0,0.0,0.0,0.0,0.0,0.0,1.0



Row sums (should be 1.0 for each state):
Class 1     1.0
Class 2     1.0
Class 3     1.0
Pass        1.0
Pub         1.0
Facebook    1.0
Sleep       1.0
dtype: float64


## Section 2: Markov Reward Process (MRP)

A **Markov Reward Process** is a tuple ⟨𝒮, ℙ, ℛ, γ⟩:

- **𝒮** is a finite set of states  
- **ℙ** is a state transition probability matrix,  
  ℙₛₛ′ = ℙ[𝑆ₜ₊₁ = s′ | 𝑆ₜ = s]  
- **ℛ** is a reward function,  
  ℛₛ = 𝔼[𝑅ₜ₊₁ | 𝑆ₜ = s]  
- **γ** is a discount factor, γ ∈ [0, 1]

Define reward vector R and solve v = (I - gamma P)^{-1} R for several gamma values.

## Value Function

The **value function** $v(s)$ gives the long-term value of state $s$.

The **state value function** $v(s)$ of a Markov Reward Process (MRP) is the expected return starting from state $s$:



$$
v(s) = \mathbb{E}[G_t \mid S_t = s]
$$

where, 

$$
G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ....
$$



## Bellman Equation for MRPs

The value function satisfies the recursive relationship:



$$
v(s) = \mathbb{E}[R_{t+1} + \gamma v(S_{t+1}) \mid S_t = s]
$$

In matrix form, 
$$
v = R + \gamma Pv
$$
 or, 
 $$
 v = (I - \gamma P)^{-1} R



In [3]:
# Reward vector
R = np.array([-2, -2, -2, 10, 1, -1, 0])

def solve_mrp(P, R, gamma):
    I = np.eye(len(P))
    try:
        v = np.linalg.inv(I - gamma * P).dot(R)
    except np.linalg.LinAlgError:
        # Use least-squares solver for singular matrices
        v = np.linalg.lstsq(I - gamma * P, R, rcond=None)[0]
    return v

gammas = [0.0, 0.9, 1.0]
results = {f'gamma={g}': solve_mrp(P, R, g) for g in gammas}

df_v = pd.DataFrame(results, index=states)
print("State-value v(s) for different gamma values:")
display(df_v)

State-value v(s) for different gamma values:


,gamma=0.0,gamma=0.9,gamma=1.0
Class 1,-2.0,-5.012729,-9.899471
Class 2,-2.0,0.942655,4.100529
Class 3,-2.0,4.087021,6.964727
Pass,10.0,10.000000,12.643739
Pub,1.0,1.908392,3.446208
Facebook,-1.0,-7.637608,-19.899471
Sleep,0.0,0.000000,2.643739


## Section 3: Markov Decision Process (MDP)

A **Markov Decision Process (MDP)** is a Markov Reward Process with decisions. It models an environment where all states satisfy the Markov property.

An MDP is defined as a tuple ⟨𝒮, 𝒜, ℙ, ℛ, γ⟩:

- **𝒮**: finite set of states  
- **𝒜**: finite set of actions  
- **ℙ**: state transition probability matrix  
  

$$
  \mathcal{P}_{ss'}^a = \mathbb{P}[S_{t+1} = s' \mid S_t = s, A_t = a]
  $$


- **ℛ**: reward function  
  

$$
  \mathcal{R}_s^a = \mathbb{E}[R_{t+1} \mid S_t = s, A_t = a]
  $$


- **γ**: discount factor, γ ∈ [0, 1]


### Section 4.1: Student MDP (Actions & Transitions)

We encode transitions as transitions[s][a] = [(prob, next_state, reward), ...].
This model follows the lecture diagram: Study, Facebook, Sleep, Pub actions as appropriate.

*Action Set:* **𝒜** = [Facebook, Quit, Study, Sleep, Pub]

![alt text](images/student_example_2.png)

In [4]:
# Define MDP transitions
# States: 0:C1, 1:C2, 2:C3, 3:Pass, 4:Pub(not used as state-action), 5:FB, 6:Sleep
transitions = {
    0: {  # Class 1
        'Study': [(1.0, 1, -2)],
        'Facebook': [(1.0, 5, -1)]
    },
    1: {  # Class 2
        'Study': [(1.0, 2, -2)],
        'Sleep': [(1.0, 6, 0)]
    },
    2: {  # Class 3
        'Study': [(1.0, 3, 10)],
        'Pub': [(0.2, 0, 1), (0.4, 1, 1), (0.4, 2, 1)]
    },
    3: {  # Pass
        'Sleep': [(1.0, 6, 0)]
    },
    5: {  # Facebook state
        'Facebook': [(1.0, 5, -1)],
        'Quit': [(1.0, 0, 0)]
    },
    6: {  # Sleep terminal
        'Sleep': [(1.0, 6, 0)]
    }
}

# Basic validation of transition probability distributions
for s, acts in transitions.items():
    for a, outs in acts.items():
        probs = [p for p,_,_ in outs]
        if not np.isclose(sum(probs), 1.0):
            print(f"Warning: probabilities for state {states[s]}, action {a} sum to {sum(probs)}")
print("MDP transitions defined and validated.")

MDP transitions defined and validated.


### Policy

A **policy** $ \pi $ is a distribution over actions given states:

$$
\pi(a \mid s) = \mathbb{P}[A_t = a \mid S_t = s]
$$

- A policy fully defines the behaviour of an agent  
- MDP policies depend only on the current state (not the history)  
- Policies are **stationary** (time-independent):  
  $$
  A_t \sim \pi(\cdot \mid S_t), \quad \forall t > 0
  $$


### State-Value Function

The **state-value function** $v_{\pi}(s)$ of an MDP is the expected return starting from state $s$, and then following policy $\pi$:

$$
v_{\pi}(s) = \mathbb{E}_{\pi} \left[ G_t \mid S_t = s \right]
$$

### Action-Value Function

The **action-value function** $q_{\pi}(s, a)$ is the expected return starting from state $s$, taking action $a$, and then following policy $\pi$:

$$
q_{\pi}(s, a) = \mathbb{E}_{\pi} \left[ G_t \mid S_t = s, A_t = a \right]
$$


### Optimal Value Function

$$
v_*(s) = max_\pi \{v_{\pi}(s)\}
$$


$$
q_*(s,a) = max_\pi \{q_{\pi}(s,a)\}
$$


In [5]:
# Value Iteration Algorithm (will be discussed in the week 2 materials)

def value_iteration(transitions, n_states, gamma=1.0, theta=1e-6, max_iters=10000):
    V = np.zeros(n_states)
    for it in range(max_iters):
        delta = 0.0
        for s in range(n_states):
            if s not in transitions:
                continue
            v = V[s]
            action_vals = []
            for a, outcomes in transitions[s].items():
                q = 0.0
                for prob, s_next, r in outcomes:
                    q += prob * (r + gamma * V[s_next])
                action_vals.append(q)
            V[s] = max(action_vals) if action_vals else 0.0
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    return V

V_star = value_iteration(transitions, n_states, gamma=1.0)
print("Optimal state values v*(s):")
display(pd.DataFrame(V_star, index=states, columns=["v*(s)"]))

# Extract greedy policy
policy = {}
for s in range(n_states):
    if s not in transitions:
        policy[states[s]] = None
        continue
    best_a = None
    best_q = -np.inf
    for a, outcomes in transitions[s].items():
        q = sum(prob * (r + 1.0 * V_star[s_next]) for prob, s_next, r in outcomes)
        if q > best_q:
            best_q = q
            best_a = a
    policy[states[s]] = best_a

print("Greedy policy from v*:")
for s, a in policy.items():
    print(f"  {s}: {a}")

Optimal state values v*(s):


,v*(s)
Class 1,6.0
Class 2,8.0
Class 3,10.0
Pass,0.0
Pub,0.0
Facebook,6.0
Sleep,0.0


Greedy policy from v*:
  Class 1: Study
  Class 2: Study
  Class 3: Study
  Pass: Sleep
  Pub: None
  Facebook: Quit
  Sleep: Sleep


## Closing notes & exercises

- Try modifying rewards and see how policies change.
- Add stochasticity to Study actions and examine robustness.
- Exercise: implement prioritized sweeping or asynchronous value iteration on this MDP.

---

End of notebook.